In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/matrix-mutations-binary-pca/matrix_mutations_binary_pca (1).csv
/kaggle/input/matrix-expression-pca/matrix_expression_pca.csv
/kaggle/input/mirrna-pca/matrix_mirRNA_pca.csv
/kaggle/input/matrix-metabolomics-pca/matrix_metabolomics_pca.csv
/kaggle/input/matrix-rppa-pca/matrix_rppa_pca.csv
/kaggle/input/matrix-cnv-pca-1/matrix_cnv_pca (1).csv


In [2]:
#Techniques inspired by: 
# https://chrisalbon.com/machine_learning/feature_engineering/select_best_number_of_components_in_tsvd/
# https://chrisalbon.com/machine_learning/feature_engineering/dimensionality_reduction_on_sparse_feature_matrix/

# Load libraries
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
from sklearn import preprocessing
from sklearn import datasets
import numpy as np

Metabolomics

In [3]:
Metabolomics_original = pd.read_csv("../input/matrix-metabolomics-pca/matrix_metabolomics_pca.csv",header=None)
metabolomics = Metabolomics_original.copy()
print(metabolomics.shape)
metabolomics.head()

(918, 226)


,0,1,2,3,4,5,6,7,8,9,...,216,217,218,219,220,221,222,223,224,225
0,SIDM00523,6.112727,6.034198,5.896896,6.000532,5.513618,5.868529,5.977177,5.693074,5.923737,...,6.070239,6.133433,6.091089,6.257711,6.372732,6.202511,5.939576,6.309821,6.115974,5.999436
1,SIDM00835,5.577413,5.727045,5.111468,6.073250,5.802494,5.824473,5.888821,5.768379,5.760784,...,6.248653,6.633575,6.378052,6.341043,6.360945,6.333540,6.137271,7.065858,6.832174,6.363064
2,SIDM00741,5.886398,5.574881,5.541259,5.848375,5.665026,5.875548,5.894904,5.839640,5.742613,...,5.942887,5.946988,5.837980,5.913350,6.137530,5.807546,5.704149,5.881193,5.785208,5.504225
3,SIDM01254,5.770030,6.099229,6.233259,5.543496,5.767759,6.155905,6.111148,5.949481,6.342703,...,6.516922,6.113791,6.282113,6.248667,6.109480,6.043570,5.846802,6.429402,5.779815,6.241530
4,SIDM00851,5.480683,5.469742,6.509397,6.251005,5.190578,5.897085,6.148333,5.607481,5.871600,...,6.161981,6.777932,6.676390,6.695659,6.751029,6.385056,6.682612,6.757899,6.728570,6.879260


In [4]:
# Remove model_id column (first column)
first_column = metabolomics.columns[0]
metabolomics = metabolomics.drop([first_column], axis=1)
metabolomics.head()

,1,2,3,4,5,6,7,8,9,10,...,216,217,218,219,220,221,222,223,224,225
0,6.112727,6.034198,5.896896,6.000532,5.513618,5.868529,5.977177,5.693074,5.923737,5.641243,...,6.070239,6.133433,6.091089,6.257711,6.372732,6.202511,5.939576,6.309821,6.115974,5.999436
1,5.577413,5.727045,5.111468,6.073250,5.802494,5.824473,5.888821,5.768379,5.760784,5.914742,...,6.248653,6.633575,6.378052,6.341043,6.360945,6.333540,6.137271,7.065858,6.832174,6.363064
2,5.886398,5.574881,5.541259,5.848375,5.665026,5.875548,5.894904,5.839640,5.742613,5.570208,...,5.942887,5.946988,5.837980,5.913350,6.137530,5.807546,5.704149,5.881193,5.785208,5.504225
3,5.770030,6.099229,6.233259,5.543496,5.767759,6.155905,6.111148,5.949481,6.342703,6.054781,...,6.516922,6.113791,6.282113,6.248667,6.109480,6.043570,5.846802,6.429402,5.779815,6.241530
4,5.480683,5.469742,6.509397,6.251005,5.190578,5.897085,6.148333,5.607481,5.871600,5.128463,...,6.161981,6.777932,6.676390,6.695659,6.751029,6.385056,6.682612,6.757899,6.728570,6.879260


In [5]:
# Load the data
X = metabolomics.iloc [:,0:225].values
print(X.shape)

(918, 225)


In [6]:
# Standardize the feature matrix
X = StandardScaler().fit_transform(X)

# Make sparse matrix
X_sparse = csr_matrix(X)
X_sparse.shape

(918, 225)

In [7]:
# Create and run an TSVD with one less than number of features
tsvd = TruncatedSVD(n_components=X_sparse.shape[1]-1)
X_tsvd = tsvd.fit(X)

In [8]:
# List of explained variances
tsvd_var_ratios = tsvd.explained_variance_ratio_

In [9]:
# Create a function
def select_n_components(var_ratio, goal_var: float) -> int:
    # Set initial variance explained so far
    total_variance = 0.0
    
    # Set initial number of features
    n_components = 0
    
    # For the explained variance of each feature:
    for explained_variance in var_ratio:
        
        # Add the explained variance to the total
        total_variance += explained_variance
        
        # Add one to the number of components
        n_components += 1
        
        # If we reach our goal level of explained variance
        if total_variance >= goal_var:
            # End the loop
            break
            
    # Return the number of components
    return n_components

In [10]:
# Run function
select_n_components(tsvd_var_ratios, 0.90)

80

In [11]:
# Create a TSVD
tsvd = TruncatedSVD(n_components=80)

In [12]:
# Conduct TSVD on sparse matrix
X_sparse_tsvd = tsvd.fit(X_sparse).transform(X_sparse)

In [13]:
# Show results
print('Original number of features:', X_sparse.shape[1])
print('Reduced number of features:', X_sparse_tsvd.shape[1])

Original number of features: 225
Reduced number of features: 80


In [14]:
print(X_sparse_tsvd)

[[-0.54429691  2.78288966  2.62296759 ... -0.94623491  1.18831362
   0.10315303]
 [-1.91309449  3.56108079  4.06897132 ... -0.20630451 -0.47154459
  -0.30304116]
 [-4.3029178   0.95863994  5.28592211 ...  0.67195882 -0.18795301
  -0.96652482]
 ...
 [-6.94643379 -0.89368361 -0.55619246 ... -0.31020464 -0.7781398
  -0.25320514]
 [-0.76663042  0.5778354  -1.04928295 ... -0.63345601 -0.71608611
   0.57715907]
 [13.52959892  9.70471107 -2.00696582 ... -0.63158905 -0.02943077
  -0.95020818]]


In [15]:
pd.DataFrame(X_sparse_tsvd).to_csv("./Meta_SVD.csv", header=None, index=None)

RPPA

In [16]:
Rppa_original = pd.read_csv("../input/matrix-rppa-pca/matrix_rppa_pca.csv",header=None)
rppa = Rppa_original.copy()
print(rppa.shape)
rppa.head()

(889, 215)


,0,1,2,3,4,5,6,7,8,9,...,205,206,207,208,209,210,211,212,213,214
0,SIDM00523,-0.104888,0.060414,0.309068,-0.075506,0.230359,0.198304,-0.030541,0.455889,0.090484,...,-0.099433,-0.486715,-1.147858,0.133876,-0.075812,-0.144388,-1.090303,-2.109324,0.178104,0.246541
1,SIDM00835,0.358504,-0.180291,-0.041237,-0.286629,-0.877406,-1.026948,-0.462761,-0.011197,0.605330,...,-0.109777,0.349330,0.770148,0.984297,-0.168138,-0.004905,0.189294,-0.283593,0.255972,-0.121134
2,SIDM00741,0.028738,0.071902,-0.094847,0.285069,1.321551,0.620703,-0.439484,0.195007,0.036221,...,0.154344,-0.478189,-1.185530,1.273013,-0.240413,0.476633,-1.367465,-2.525695,-0.137880,-0.451282
3,SIDM01254,0.120039,-0.066802,-0.128007,-0.552081,-0.292428,-1.415935,-0.138858,-0.066122,-0.346564,...,0.040106,5.923830,-3.893832,-2.499188,0.632758,0.025639,-1.189180,-3.056863,0.025997,-0.465205
4,SIDM00851,-0.268997,-0.060281,-0.137881,-0.398729,-0.095622,-0.533905,0.054245,-0.573022,-0.162968,...,-0.466919,5.475880,-0.561973,-0.500953,-0.261494,0.358679,-0.951686,-3.247388,-0.151424,-0.145426


In [17]:
# Remove model_id column (first column)
first_column = rppa.columns[0]
rppa = rppa.drop([first_column], axis=1)
rppa.head()

,1,2,3,4,5,6,7,8,9,10,...,205,206,207,208,209,210,211,212,213,214
0,-0.104888,0.060414,0.309068,-0.075506,0.230359,0.198304,-0.030541,0.455889,0.090484,0.268448,...,-0.099433,-0.486715,-1.147858,0.133876,-0.075812,-0.144388,-1.090303,-2.109324,0.178104,0.246541
1,0.358504,-0.180291,-0.041237,-0.286629,-0.877406,-1.026948,-0.462761,-0.011197,0.605330,-0.614974,...,-0.109777,0.349330,0.770148,0.984297,-0.168138,-0.004905,0.189294,-0.283593,0.255972,-0.121134
2,0.028738,0.071902,-0.094847,0.285069,1.321551,0.620703,-0.439484,0.195007,0.036221,0.767920,...,0.154344,-0.478189,-1.185530,1.273013,-0.240413,0.476633,-1.367465,-2.525695,-0.137880,-0.451282
3,0.120039,-0.066802,-0.128007,-0.552081,-0.292428,-1.415935,-0.138858,-0.066122,-0.346564,0.586421,...,0.040106,5.923830,-3.893832,-2.499188,0.632758,0.025639,-1.189180,-3.056863,0.025997,-0.465205
4,-0.268997,-0.060281,-0.137881,-0.398729,-0.095622,-0.533905,0.054245,-0.573022,-0.162968,-0.691478,...,-0.466919,5.475880,-0.561973,-0.500953,-0.261494,0.358679,-0.951686,-3.247388,-0.151424,-0.145426


In [18]:
X = rppa.iloc [:,0:214].values
print(X.shape)

(889, 214)


In [19]:
# Standardize the feature matrix
X = StandardScaler().fit_transform(X)

# Make sparse matrix
X_sparse = csr_matrix(X)
X_sparse.shape

# Create and run an TSVD with one less than number of features
tsvd = TruncatedSVD(n_components=X_sparse.shape[1]-1)
X_tsvd = tsvd.fit(X)

# List of explained variances
tsvd_var_ratios = tsvd.explained_variance_ratio_


In [20]:
# Create a function
def select_n_components(var_ratio, goal_var: float) -> int:
    # Set initial variance explained so far
    total_variance = 0.0
    
    # Set initial number of features
    n_components = 0
    
    # For the explained variance of each feature:
    for explained_variance in var_ratio:
        
        # Add the explained variance to the total
        total_variance += explained_variance
        
        # Add one to the number of components
        n_components += 1
        
        # If we reach our goal level of explained variance
        if total_variance >= goal_var:
            # End the loop
            break
            
    # Return the number of components
    return n_components

In [21]:
# Run function
select_n_components(tsvd_var_ratios, 0.90)

101

In [22]:
# Create a TSVD
tsvd = TruncatedSVD(n_components=101)
# Conduct TSVD on sparse matrix
X_sparse_tsvd = tsvd.fit(X_sparse).transform(X_sparse)

# Show results
print('Original number of features:', X_sparse.shape[1])
print('Reduced number of features:', X_sparse_tsvd.shape[1])


Original number of features: 214
Reduced number of features: 101


In [23]:
print(X_sparse_tsvd)
pd.DataFrame(X_sparse_tsvd).to_csv("./RPPA_SVD.csv", header=None, index=None)

[[ 1.93489036 -0.63607047  2.48654503 ... -1.20480197 -1.49032602
   0.93525746]
 [-6.54327181 11.26118995  5.82239339 ... -1.19687787 -0.32457657
  -0.36389042]
 [ 7.3078695  -1.75885832 -2.48143792 ...  0.09514057  0.61065469
   1.2254688 ]
 ...
 [-3.3774837   2.55207071 -4.92769087 ...  0.62715467  1.16583141
   0.02082623]
 [-2.532423   -4.51757082 -5.27967175 ...  0.47856657  0.01211702
  -0.55807665]
 [-3.36059805 -4.06848134  4.14527287 ... -0.94254849  1.73038
   1.04238303]]


Expression

In [24]:
Expression_original = pd.read_csv("../input/matrix-expression-pca/matrix_expression_pca.csv",header=None)
expression = Expression_original.copy()
print(expression.shape)
expression.head()

(1257, 17360)


,0,1,2,3,4,5,6,7,8,9,...,17350,17351,17352,17353,17354,17355,17356,17357,17358,17359
0,SIDM01737,0.000000,0.0,4.667324,1.761285,3.554589,4.358959,0.641546,2.201634,1.941106,...,0.000000,0.0,1.565597,0.0,0.0,0.0,0.0,0.0,5.132988,0.443607
1,SIDM01094,0.000000,0.0,6.198887,2.032101,3.755956,2.601697,0.000000,0.000000,3.415488,...,0.028569,0.0,1.778209,0.0,0.0,0.0,0.0,0.0,4.100978,2.589763
2,SIDM01795,4.934988,0.0,6.635464,1.929791,2.503349,0.000000,4.883621,6.772546,3.889474,...,0.000000,0.0,1.910733,0.0,0.0,0.0,0.0,0.0,4.288359,0.475085
3,SIDM00413,0.839960,0.0,5.376082,2.687061,4.440288,4.938286,0.536053,3.584963,2.935460,...,0.659925,0.0,2.653060,0.0,0.0,0.0,0.0,0.0,4.841470,3.904966
4,SIDM00859,1.744161,0.0,6.846744,2.147307,3.666757,5.252098,0.014355,1.641546,4.100978,...,0.084064,0.0,0.150560,0.0,0.0,0.0,0.0,0.0,4.187451,1.469886


In [25]:
# Remove model_id column (first column)
first_column = expression.columns[0]
expression = expression.drop([first_column], axis=1)
expression.head()

,1,2,3,4,5,6,7,8,9,10,...,17350,17351,17352,17353,17354,17355,17356,17357,17358,17359
0,0.000000,0.0,4.667324,1.761285,3.554589,4.358959,0.641546,2.201634,1.941106,2.150560,...,0.000000,0.0,1.565597,0.0,0.0,0.0,0.0,0.0,5.132988,0.443607
1,0.000000,0.0,6.198887,2.032101,3.755956,2.601697,0.000000,0.000000,3.415488,3.620586,...,0.028569,0.0,1.778209,0.0,0.0,0.0,0.0,0.0,4.100978,2.589763
2,4.934988,0.0,6.635464,1.929791,2.503349,0.000000,4.883621,6.772546,3.889474,4.139961,...,0.000000,0.0,1.910733,0.0,0.0,0.0,0.0,0.0,4.288359,0.475085
3,0.839960,0.0,5.376082,2.687061,4.440288,4.938286,0.536053,3.584963,2.935460,3.157044,...,0.659925,0.0,2.653060,0.0,0.0,0.0,0.0,0.0,4.841470,3.904966
4,1.744161,0.0,6.846744,2.147307,3.666757,5.252098,0.014355,1.641546,4.100978,4.246408,...,0.084064,0.0,0.150560,0.0,0.0,0.0,0.0,0.0,4.187451,1.469886


In [26]:
X = expression.iloc [:,0:17359].values
print(X.shape)

(1257, 17359)


In [27]:
# Standardize the feature matrix
X = StandardScaler().fit_transform(X)

# Make sparse matrix
X_sparse = csr_matrix(X)
X_sparse.shape

# Create and run an TSVD with one less than number of features
tsvd = TruncatedSVD(n_components=X_sparse.shape[1]-1)
X_tsvd = tsvd.fit(X)

# List of explained variances
tsvd_var_ratios = tsvd.explained_variance_ratio_


In [28]:
# Create a function
def select_n_components(var_ratio, goal_var: float) -> int:
    # Set initial variance explained so far
    total_variance = 0.0
    
    # Set initial number of features
    n_components = 0
    
    # For the explained variance of each feature:
    for explained_variance in var_ratio:
        
        # Add the explained variance to the total
        total_variance += explained_variance
        
        # Add one to the number of components
        n_components += 1
        
        # If we reach our goal level of explained variance
        if total_variance >= goal_var:
            # End the loop
            break
            
    # Return the number of components
    return n_components

In [29]:
# Run function
select_n_components(tsvd_var_ratios, 0.90)

616

In [30]:
# Create a TSVD
tsvd = TruncatedSVD(n_components=616)
# Conduct TSVD on sparse matrix
X_sparse_tsvd = tsvd.fit(X_sparse).transform(X_sparse)

# Show results
print('Original number of features:', X_sparse.shape[1])
print('Reduced number of features:', X_sparse_tsvd.shape[1])

Original number of features: 17359
Reduced number of features: 616


In [31]:
print(X_sparse_tsvd)
pd.DataFrame(X_sparse_tsvd).to_csv("./Exp_SVD.csv", header=None, index=None)

[[ 1.25843839e+02  8.22311661e+01 -2.85411605e+01 ...  1.63788030e+00
  -7.24485298e-01 -1.52130041e+00]
 [ 5.14220666e+01  9.08408766e+01 -2.34153247e+01 ... -1.48923063e+00
  -4.96127451e+00  3.51231935e+00]
 [ 2.90163212e+01 -4.14287771e+01 -6.53567404e+01 ...  1.16756826e+00
   2.60057818e+00  5.19165980e-02]
 ...
 [ 1.55948097e+01 -3.15587029e+01  4.70192124e+01 ... -7.56764786e-01
   3.16885664e-01  5.81653442e-01]
 [ 1.19053654e+01 -9.26000092e+00  4.92536325e+01 ...  3.19834613e-01
  -4.25730854e+00 -5.62884817e-01]
 [-5.43120887e+01  6.80938234e+00 -1.06458036e+01 ...  2.55950742e+00
  -5.39609070e+00 -3.29187396e+00]]


Mutations

In [32]:
Mutations_original = pd.read_csv("../input/matrix-mutations-binary-pca/matrix_mutations_binary_pca (1).csv",header=None)
mutations = Mutations_original.copy()
print(mutations.shape)
mutations.head()

(1645, 18771)


,0,1,2,3,4,5,6,7,8,9,...,18761,18762,18763,18764,18765,18766,18767,18768,18769,18770
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
X = mutations.iloc [:,0:18770].values
print(X.shape)

(1645, 18770)


In [34]:
# Standardize the feature matrix
X = StandardScaler().fit_transform(X)

# Make sparse matrix
X_sparse = csr_matrix(X)
X_sparse.shape

# Create and run an TSVD with one less than number of features
tsvd = TruncatedSVD(n_components=X_sparse.shape[1]-1)
X_tsvd = tsvd.fit(X)

# List of explained variances
tsvd_var_ratios = tsvd.explained_variance_ratio_


In [35]:
# Create a function
def select_n_components(var_ratio, goal_var: float) -> int:
    # Set initial variance explained so far
    total_variance = 0.0
    
    # Set initial number of features
    n_components = 0
    
    # For the explained variance of each feature:
    for explained_variance in var_ratio:
        
        # Add the explained variance to the total
        total_variance += explained_variance
        
        # Add one to the number of components
        n_components += 1
        
        # If we reach our goal level of explained variance
        if total_variance >= goal_var:
            # End the loop
            break
            
    # Return the number of components
    return n_components

In [36]:
# Run function
select_n_components(tsvd_var_ratios, 0.90)

1040

In [37]:
# Create a TSVD
tsvd = TruncatedSVD(n_components=1040)
# Conduct TSVD on sparse matrix
X_sparse_tsvd = tsvd.fit(X_sparse).transform(X_sparse)

# Show results
print('Original number of features:', X_sparse.shape[1])
print('Reduced number of features:', X_sparse_tsvd.shape[1])

Original number of features: 18770
Reduced number of features: 1040


In [38]:
print(X_sparse_tsvd)
pd.DataFrame(X_sparse_tsvd).to_csv("./Mut_SVD.csv", header=None, index=None)

[[-12.3973047    1.69128159  -1.24658999 ...  -7.13238285  -5.2162088
    4.01414629]
 [ -7.12091845  -0.6941931    1.12104465 ...  -2.68293242   0.55092746
    2.32765661]
 [ 28.40315993 -17.29860345  13.90482712 ...   1.21967603   0.40968458
    0.78673869]
 ...
 [ -0.74633983  -3.0610964    3.17883344 ...  -0.22753473  -0.61238674
    1.1166836 ]
 [ -0.19153452  -2.98681331   3.23220884 ...  -0.7255335   -0.62125763
    1.10239294]
 [ -0.22571059  -2.44484676   3.45539531 ...  -0.81538457  -0.72450466
    0.30591315]]


miRNA

In [39]:
miRNA_original = pd.read_csv("../input/mirrna-pca/matrix_mirRNA_pca.csv",header=None)
miRNA = miRNA_original.copy()
print(miRNA.shape)
miRNA.head()

(942, 735)


,0,1,2,3,4,5,6,7,8,9,...,725,726,727,728,729,730,731,732,733,734
0,SIDM00523,4362.58,187.44,267.03,868.11,1.04,959.22,1638.85,1229.40,10.48,...,7.33,48.17,12.56,23.04,57.60,13.61,4.19,27.23,9.42,37.69
1,SIDM00835,5191.50,868.22,244.88,556.55,92.02,1519.75,1067.09,553.58,25.23,...,10.39,47.49,20.78,40.07,62.33,34.14,13.36,59.37,13.36,80.14
2,SIDM00741,24991.05,5066.09,818.49,3661.86,942.62,4616.12,2195.57,4243.72,15.52,...,3.88,38.79,11.64,23.27,58.19,17.45,5.81,44.60,7.76,52.36
3,SIDM01254,3253.83,74.21,53.28,315.87,1.90,755.43,1162.62,388.18,45.67,...,7.61,58.99,15.23,119.88,178.87,15.23,5.71,34.25,7.61,125.58
4,SIDM00851,225.28,35.74,27.19,94.00,0.77,147.60,183.33,231.50,12.43,...,4.66,43.50,18.64,30.29,71.47,15.54,8.54,17.87,9.33,17.09


In [40]:
# Remove model_id column (first column)
first_column = miRNA.columns[0]
miRNA = miRNA.drop([first_column], axis=1)
miRNA.head()

,1,2,3,4,5,6,7,8,9,10,...,725,726,727,728,729,730,731,732,733,734
0,4362.58,187.44,267.03,868.11,1.04,959.22,1638.85,1229.40,10.48,7.33,...,7.33,48.17,12.56,23.04,57.60,13.61,4.19,27.23,9.42,37.69
1,5191.50,868.22,244.88,556.55,92.02,1519.75,1067.09,553.58,25.23,32.65,...,10.39,47.49,20.78,40.07,62.33,34.14,13.36,59.37,13.36,80.14
2,24991.05,5066.09,818.49,3661.86,942.62,4616.12,2195.57,4243.72,15.52,17.45,...,3.88,38.79,11.64,23.27,58.19,17.45,5.81,44.60,7.76,52.36
3,3253.83,74.21,53.28,315.87,1.90,755.43,1162.62,388.18,45.67,91.34,...,7.61,58.99,15.23,119.88,178.87,15.23,5.71,34.25,7.61,125.58
4,225.28,35.74,27.19,94.00,0.77,147.60,183.33,231.50,12.43,12.43,...,4.66,43.50,18.64,30.29,71.47,15.54,8.54,17.87,9.33,17.09


In [41]:
X = miRNA.iloc [:,0:734].values
print(X.shape)

(942, 734)


In [42]:
# Standardize the feature matrix
X = StandardScaler().fit_transform(X)

# Make sparse matrix
X_sparse = csr_matrix(X)
X_sparse.shape

# Create and run an TSVD with one less than number of features
tsvd = TruncatedSVD(n_components=X_sparse.shape[1]-1)
X_tsvd = tsvd.fit(X)

# List of explained variances
tsvd_var_ratios = tsvd.explained_variance_ratio_

In [43]:
# Create a function
def select_n_components(var_ratio, goal_var: float) -> int:
    # Set initial variance explained so far
    total_variance = 0.0
    
    # Set initial number of features
    n_components = 0
    
    # For the explained variance of each feature:
    for explained_variance in var_ratio:
        
        # Add the explained variance to the total
        total_variance += explained_variance
        
        # Add one to the number of components
        n_components += 1
        
        # If we reach our goal level of explained variance
        if total_variance >= goal_var:
            # End the loop
            break
            
    # Return the number of components
    return n_components

In [44]:
# Run function
select_n_components(tsvd_var_ratios, 0.90)

197

In [45]:
# Create a TSVD
tsvd = TruncatedSVD(n_components=197)
# Conduct TSVD on sparse matrix
X_sparse_tsvd = tsvd.fit(X_sparse).transform(X_sparse)

# Show results
print('Original number of features:', X_sparse.shape[1])
print('Reduced number of features:', X_sparse_tsvd.shape[1])

Original number of features: 734
Reduced number of features: 197


In [46]:
print(X_sparse_tsvd)
pd.DataFrame(X_sparse_tsvd).to_csv("./miRNA_SVD.csv", header=None, index=None)

[[-1.06028511e+01 -8.54666288e-01 -4.81936178e-01 ... -4.93838439e-01
   1.13149543e-02  3.93584459e-01]
 [ 9.39918437e+00 -9.49505525e+00  1.25583918e+00 ... -1.23637613e+00
   2.38321083e-01  2.89461797e-01]
 [ 6.67027537e+00  5.59840624e+00 -1.34104591e-01 ...  6.23953600e-02
  -6.96102690e-01  4.89164438e-01]
 ...
 [-1.95898894e+00  1.49336411e+01  1.64343039e+01 ... -3.40047157e-02
   2.94392116e-01 -1.50947781e-01]
 [-1.41641713e+01  8.21292620e-01 -4.56991091e-01 ... -6.52044575e-01
  -2.17602945e-01 -9.02831486e-02]
 [-1.26108514e+01 -2.25313820e+00 -3.34568796e-01 ...  4.34444200e-01
   1.28668165e+00 -2.86663274e-01]]
